# 1.Top_Skills_Score

There will be more chance of getting hired if the candidate has top demanding skills.

Scoring be like:

0.1 - the candidate have 1 out top 10 skills

0.2 - the candidate have 2 out top 10 skills

likewise ............

In [5]:
import pandas as pd
import numpy as np

train = pd.read_csv('S:/Hackathons/TCS/train1.csv')
skills_list = pd.read_csv('S:/Hackathons/TCS/skills_list.csv')
name_skills_df = pd.read_csv('S:/Hackathons/TCS/datasets/name_skills_df1')

name_skills_df.drop_duplicates(inplace = True)

In [6]:
selected_cands = list(train[train['evaluation_status'] == 'SELECTED']['can_id'])


selected_can_inv_skills = name_skills_df[name_skills_df['name'].isin(selected_cands)]
selected_can_ct = pd.crosstab(selected_can_inv_skills['name'], selected_can_inv_skills['skill'],margins = True)


num_skills_sel = pd.DataFrame(index = selected_can_ct.columns)
num_skills_sel['#_sel_skills'] = list(selected_can_ct.loc[['All']].values[0])

num_skills_sel = num_skills_sel.sort_values(by = ['#_sel_skills'],ascending = False)
num_skills_sel.drop('All',inplace =True)

In [7]:
num_skills_sel.head(10)

,#_sel_skills
skill,
python,313
c++,294
java,252
web development,137
ai/ml,116
sql,100
javascript,88
dsa,84
css3,70


In [8]:
top_skills = list(num_skills_sel.index[0:11])




id_vs_skills_one_hot = pd.crosstab(name_skills_df['name'], name_skills_df['skill'],margins = True)
id_vs_skills_one_hot.reset_index(inplace = True)

id_vs_skills_one_hot[['name']+top_skills]

top_skills_score = id_vs_skills_one_hot[['name']+top_skills].sum(axis = 1, skipna = True)*0.1
id_vs_skills_one_hot['top_skills_score'] = top_skills_score

top_skills_score = id_vs_skills_one_hot[['name','top_skills_score']]
top_skills_score.head(7)

skill,name,top_skills_score
0,c1,0.3
1,c10,0.6
2,c100,0.2
3,c101,0.4
4,c102,0.5
5,c104,0.2
6,c105,0.3


In [9]:
#top_skills_score.to_csv('S:/Hackathons/TCS/top_skills_score.csv',index = False)

In [15]:
id_vs_skills_one_hot.head(10)

skill,abby & python,adf,adobe aftereffects,adobe illustrator,adobe photoshop,adobe photoshop ui ux,adobe premier pro,adobe spark,adobe xd ui ux,advanced java,...,whatsapp coding,windows,wireshark,wordpress,xamarin,xampp,xml,zk,All,top_skills_score
name,,,,,,,,,,,,,,,,,,,,,
c1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0.3
c10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,0.6
c100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0.2
c101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0.4
c102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0.5
c104,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,7,0.2
c105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0.3
c106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,0.4
c107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0.3



# 2.id lookup 

Both candidate and project variables in train data have high cardinality.
One way of dealing with high cardinality features is by producing LookUp Table

The Lookup's can be produced by many methods like from

1.Embedding Layer of Keras.

2.Encoding from other features of dataset.

3.....

For this case The best way of representing the skills of candidate and project id OneHotEncoding as you see above

Clearly, OnHotEncoded dataframe id_vs_skills_one_hot has high sparsity.We can apply Singular Value Decomposition on this high 
sparsity data and use latent features of candidate_id and project_id as a lookup table

In [12]:

from scipy.sparse.linalg import svds
id_vs_skills_one_hot.index = id_vs_skills_one_hot.name
id_vs_skills_one_hot.drop(columns = ['name'],inplace = True)

matrix = id_vs_skills_one_hot
matrix = matrix.astype(float)
U, sigma, Vt = svds(matrix)

U = pd.DataFrame(U,columns = ['u1','u2','u3','u4','u5','u6'])

U.index = id_vs_skills_one_hot.index
U

,u1,u2,u3,u4,u5,u6
name,,,,,,
c1,-0.016077,-0.018801,0.018124,-0.009466,0.039051,0.000965
c10,0.014980,0.015430,-0.030699,-0.066464,0.015383,0.001169
c100,0.009704,0.015187,0.036413,-0.024555,0.026160,0.000582
c101,0.006896,0.014065,0.031490,-0.031910,0.000887,0.000953
c102,0.023480,0.011398,-0.034634,0.048481,0.022848,0.000963
...,...,...,...,...,...,...
p96,-0.004040,-0.016086,0.033383,0.022639,-0.042222,0.000903
p97,0.011080,0.021276,0.031486,-0.028202,0.033910,0.000401
p98,-0.048261,0.016455,0.067491,0.001982,-0.027921,0.001103


In [ ]:
#U = U.to_csv('S:/Hackathons/TCS/id_lookup.csv',index = True)